Python program to parse PL/1 and COBOL programs and extract subroutine calls

In [325]:
# step 1 : Read first/next module name from an excel file
# step 2 : Detect language
# step 3 : Find subroutines for PLI_BATCH, PLI_ONLINE, PLI_SLC, COB-BATCH, COB-ONLINE
# step 4 : Append module names to the excel file
# step 5 : Goto step 1 (until end of file)
# step 6 : Generate report (a.all modules list and b.tree structure of modules and subroutines)

In [326]:
file_name = 'C:\\Users\\raVan\\MainframeParser\\PLIsample.txt'

In [327]:
# 1. Detect language (PLI / COBOL), MODE (BATCH / ONLINE / SLC)

def detect_language():
    print('# entered detect_language()')
    # respective flags for above mentioned detections
    COB_PGM_FLG = 'N'
    PLI_PGM_FLG = 'N'
    BAT_PGM_FLG = 'N'
    ONL_PGM_FLG = 'N'
    SLC_PGM_FLG = 'N'

    #print('COB_PGM_FLG | PLI_PGM_FLG | BAT_PGM_FLG | ONL_PGM_FLG | SLC_PGM_FLG') 
    #print('     ',COB_PGM_FLG,'          ',PLI_PGM_FLG,'           ',BAT_PGM_FLG,'           ',ONL_PGM_FLG,'           ',SLC_PGM_FLG) 

    
    with open(file_name,'r') as f:
        for line in f:
            if 'IDENTIFICATION DIVISION.' in line or 'PROCEDURE DIVISION.' in line:
                COB_PGM_FLG = 'Y'
            if 'DCL' in line or 'PUT SKIP' in line:
                PLI_PGM_FLG = 'Y'    
            if 'SSBCICS_SW' in line:
                SLC_PGM_FLG = 'Y'
            if 'EXEC CICS LINK' in line or 'EXEC CICS LOAD' in line or 'EXEC CICS XCTL' in line:
                ONL_PGM_FLG = 'Y'
            else:
                BAT_PGM_FLG = 'Y'
            
    print('COB_PGM_FLG | PLI_PGM_FLG | BAT_PGM_FLG | ONL_PGM_FLG | SLC_PGM_FLG') 
    print('     ',COB_PGM_FLG,'          ',PLI_PGM_FLG,'           ',BAT_PGM_FLG,'           ',ONL_PGM_FLG,'           ',SLC_PGM_FLG) 
    
    find_subs(COB_PGM_FLG,PLI_PGM_FLG,BAT_PGM_FLG,ONL_PGM_FLG,SLC_PGM_FLG)

In [328]:
def find_subs(COB_PGM_FLG,PLI_PGM_FLG,BAT_PGM_FLG,ONL_PGM_FLG,SLC_PGM_FLG):
    print('# entered find_subs()')
    count=0
    module_list = []

    with open(file_name,'r') as f:
        for line in f:
            if PLI_PGM_FLG == 'Y' and BAT_PGM_FLG == 'Y' :      
                # entries in PLI BATCH program
                if 'ENTRY' in line and 'DCL' in line :
                        print(line)
                        module_name = list(line.split())
                        module_name = module_name[module_name.index('DCL') + 1]
                        module_list.append(module_name)
            if COB_PGM_FLG == 'Y' and BAT_PGM_FLG == 'Y' :      
                # calls in COBOL BATCH program
                if 'CALL' in line and 'USING' in line :
                        print(line)
                        module_name = list(line.split())
                        module_name = module_name[module_name.index('CALL') + 1]
                        module_name = module_name.replace("'","")
                        module_list.append(module_name)
            if (PLI_PGM_FLG == 'Y' and ONL_PGM_FLG == 'Y') or (COB_PGM_FLG == 'Y' and ONL_PGM_FLG == 'Y') :      
                # calls in COBOL / PLI online program
                if 'EXEC CICS' in line and 'PROGRAM' in line :
                        print(line)
                        module_name = list(line.split())
                        module_name = module_name[module_name.index('PROGRAM') + 1]
                        module_name = module_name.replace("'","")
                        module_name = module_name.replace('(','')
                        module_name = module_name.replace(')','')
                        module_list.append(module_name)

                count+=1
        
    print('Total lines in input program :',count)
    print(module_list)

In [329]:
detect_language()

# entered detect_language()
COB_PGM_FLG | PLI_PGM_FLG | BAT_PGM_FLG | ONL_PGM_FLG | SLC_PGM_FLG
      N            Y             Y             N             N
# entered find_subs()
 DCL GIMAPI ENTRY(CHAR(8),PTR,PTR,CHAR(3),FIXED BIN(31),

 DCL ABCDEFGH                ENTRY(*)

 DCL IJKLMNOP         ENTRY(*)

 DCL LMNOPQRS    ENTRY(*)

 DCL ENTRYSTR   CHAR(100);

  DCL CURENTRY    POINTER;

Total lines in input program : 0
['GIMAPI', 'ABCDEFGH', 'IJKLMNOP', 'LMNOPQRS', 'ENTRYSTR', 'CURENTRY']
